In [8]:
import numpy as np
import matplotlib.pyplot as plt
def machineEpsilon(func=np.float64):
    machine_epsilon = func(1)
    while func(1)+machine_epsilon != func(1):
        machine_epsilon_last = machine_epsilon
        machine_epsilon = func(machine_epsilon) / func(2)
    return machine_epsilon_last

sqeps = np.sqrt(machineEpsilon())

print(np.finfo(float).eps)

2.220446049250313e-16


In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

x = np.array([0, 25, 50, 75, 100, 125, 150, 175, 200])
y = np.array([10.6, 16.0, 45.0, 83.5, 52.8, 19.9, 10.8, 8.25, 4.7])
rho = np.array([2.3, 3.5, 4.5, 6.4, 4.4, 3.4, 2.1, 1.6, 1.1])

#starting valuse
a = np.array([60000, 70, 750])

#fitting function breit wigner
def f(x, a):
    return a[0]/((a[1] - x)**2 + a[2])

#function to calculate the chi2
def chi2(x, y, rho, a):
    return np.sum(((y - f(x, a))/rho)**2)

#beta

def beta(chi2, x, y, a, rho, h=sqeps):
    j = np.zeros(len(a))
    ak = np.zeros(len(a))
    for i in range(len(a)):
        ak[i] = np.sum((y - f(x, a))/rho**2 * (f(x, a+ h*a[i] * np.eye(len(a))[i]) - f(x, a - h*a[i] * np.eye(len(a))[i]))/(2*h*a[i]))
    return ak

# alpha

def alpha(chi2, x, y, a, rho, h=sqeps):
    ap = np.zeros((len(a), len(a)))
    for i in range(len(a)):
        for j in range(len(a)):
            ap[i, j] = np.sum(((f(x, a + h*a[i] * np.eye(len(a))[i]) - f(x, a - h*a[i] * np.eye(len(a))[i]))/(2*h*a[i]) * (f(x, a + h*a[i] * np.eye(len(a))[j]) - f(x, a - h*a[i] * np.eye(len(a))[j]))/(2*h*a[i])/rho**2))
    return ap

#newton step

def newton_step(chi2, x, y, a, rho, h=sqeps):
    anew = a + np.linalg.pinv(alpha(chi2, x, y, a, rho, h)) @ beta(chi2, x, y, a, rho, h)
    return anew

print(alpha(chi2, x, y, a, rho))
print(beta(chi2, x, y, a, rho))


[[ 1.29459385e-07 -1.25684743e-05 -6.23420097e-06]
 [-1.25681185e-05  3.62391850e-01  4.21734690e-04]
 [-6.23420038e-06  4.21734511e-04  3.77938793e-04]]
[ 5.65729703e-04  2.66545297e+00 -1.67253945e-02]


In [10]:

chilist = []
alist = []
for i in range(3):
    a = newton_step(chi2, x, y, a, rho)
    alist.append(a)
    chilist.append(chi2(x, y, rho, a))

for i in range(100):
    a = newton_step(chi2, x, y, a, rho)
    chilist.append(chi2(x, y, rho, a))
    alist.append(a)
    # if np.abs(chilist[-1] - chilist[-2]) <= np.abs(chilist[-2] * 1e-6):
    #     break

#return minimum chi2
print(min([abs(i) for i in chilist]))


print(alist[[abs(i) for i in chilist].index(min([abs(i) for i in chilist]))])


print(chilist)
print(a)



2.7023271328659364
[68310.00237878    77.57646324   807.87147937]
[3.4070532099832804, 2.7069717182821997, 2.7023278182108417, 2.7023271329368144, 2.7023271328659484, 2.7023271328659417, 2.702327132865939, 2.7023271328659395, 2.7023271328659413, 2.7023271328659404, 2.702327132865941, 2.70232713286594, 2.702327132865942, 2.702327132865943, 2.702327132865941, 2.7023271328659417, 2.7023271328659404, 2.702327132865945, 2.7023271328659417, 2.7023271328659413, 2.702327132865941, 2.7023271328659417, 2.7023271328659417, 2.7023271328659413, 2.7023271328659417, 2.7023271328659435, 2.702327132865941, 2.7023271328659435, 2.702327132865939, 2.702327132865943, 2.70232713286594, 2.7023271328659404, 2.7023271328659435, 2.7023271328659386, 2.702327132865939, 2.702327132865942, 2.702327132865941, 2.702327132865941, 2.7023271328659395, 2.7023271328659404, 2.702327132865939, 2.7023271328659377, 2.702327132865941, 2.70232713286594, 2.7023271328659417, 2.702327132865942, 2.70232713286594, 2.702327132865939,

In [13]:
#levenberg step
a = np.array([-1, 1, 1])
alev = []
chilev = []
def ltep(chi2, x, y, a, rho, lam, h=sqeps):
    anew = a + (np.linalg.pinv(alpha(chi2, x, y, a, rho, h) + lam * np.diag(np.diag(alpha(chi2, x, y, a, rho, h)))) @ beta(chi2, x, y, a, rho, h))
    return anew



lam = 1.0


for i in range(1000):
    anew = ltep(chi2, x, y, a, rho, lam)
    print(f"step {i}")
    print(chi2(x, y, rho, a), chi2(x, y, rho, anew))
    

    if abs(chi2(x, y, rho, anew)) < abs(chi2(x, y, rho, a)):
        a = anew
        lam = lam/10
        print(lam)
        alev.append(a)
        chilev.append(chi2(x, y, rho, a))
        
    else:
        lam = lam*10
        print(lam)
        if lam > 1e20:
            break

print(min([abs(i) for i in chilev]))
print(alev[[abs(i) for i in chilev].index(min([abs(i) for i in chilev]))])

print(chilev)
print(alev)

step 0
563.9675664712113 561.5143708316485
0.1
step 1
561.5143708316485 559.1029387968879
0.01
step 2
559.1029387968879 560.6445598518777
0.1
step 3
559.1029387968879 550.6532713038422
0.01
step 4
550.6532713038422 561.9079190067124
0.1
step 5
550.6532713038422 561.8352806705444
1.0
step 6
550.6532713038422 561.1759813783843
10.0
step 7
550.6532713038422 554.9311484111602
100.0
step 8
550.6532713038422 530.859118405988
10.0
step 9
530.859118405988 556.1310750579132
100.0
step 10
530.859118405988 522.8216979408974
10.0
step 11
522.8216979408974 4612.360243264115
100.0
step 12
522.8216979408974 510.1721655549258
10.0
step 13
510.1721655549258 407.51785381797106
1.0
step 14
407.51785381797106 257.40904669200694
0.1
step 15
257.40904669200694 2024.3536980472402
1.0
step 16
257.40904669200694 164.2377724504794
0.1
step 17
164.2377724504794 277.9432262942177
1.0
step 18
164.2377724504794 140.8298261382922
0.1
step 19
140.8298261382922 57.59098852939686
0.01
step 20
57.59098852939686 2.896077